### Gender Classification using Logistic Regression Model

The main objective of this notebook is to see how logistic regression is used together along with a gender classification dataset that contains about 58,000 male and female images that are 

### Requirements from Rubric (dont delete muna)

An overview or description of the data is provided, including how it was collected, and its implications on the types of conclusions that could be made from the data. A description of the variables, observations, and/or structure of the data is provided. The target task is well introduced and clearly defined.

The data is sufficiently explored to get a grasp of the distribution and the content of the data. Appropriate summaries and visualizations are presented. Insights into how the EDA can help the model training is mentioned.

The necessary steps for preprocessing and cleaning are performed, including explanations for every step. If no preprocessing or cleaning is done, there is a justification on why it was not needed.

The appropriate models are used to accomplish the machine learning task. Justification of choosing the models is shown.

Appropriate data-driven error analysis is made, and changes to the model selection and hyperparameters are performed to improve model performance. The study exhausts improvements that can be done to the model

The study is concluded by effectively summarizing the efforts of the authors. Recommendations on how the model could be further improved are provided.

### About the Dataset

The Gender Classification Dataset 

In [43]:
from torchvision import transforms

# Image transformations
image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ]),
    # Validation does not use augmentation
    'valid':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

from torchvision import datasets
from torch.utils.data import DataLoader
import os

# Datasets from folders

# tm_path = "C:\Users\uyjus\Documents\GitHub\STINTSY-FinalProject-Gender-Classification\train\female"

data = {
    'train_male':
    datasets.ImageFolder(root='train/train_female', transform=image_transforms['train']),
    'train_female':
    datasets.ImageFolder(root='train/train_male', transform=image_transforms['train']),
    'validation_male':
    datasets.ImageFolder(root='valid/valid_male', transform=image_transforms['valid']),
    'validation_female':
    datasets.ImageFolder(root='valid/valid_female', transform=image_transforms['valid']),
}


tm_batch_size=len(data["train_male"])
print (tm_batch_size)
tf_batch_size=len(data["train_female"])
print (tf_batch_size)
vm_vatch_size=len(data["validation_male"])
print (vm_vatch_size)
vf_batch_size=len(data["validation_female"])
print (vf_batch_size)

# # Dataloader iterators, make sure to shuffle
# dataloaders = {
#     'train_male': DataLoader(data['train_male'], batch_size=tm_batch_size, shuffle=True),
#     'train_female': DataLoader(data['train_female'], batch_size=tf_batch_size, shuffle=True),
#     'validation_male': DataLoader(data['validation_male'], batch_size=vm_batch_size, shuffle=True),
#     'validation_female': DataLoader(data['validation_female'], batch_size=vf_batch_size, shuffle=True)
# }

23243
23766
5808
5841
